Operations

In [1]:
def invert(x):
    # x = x0 + x1*epsilon
    x0, x1 = x.list()
    assert x0 != 0  # otherwise not invertible
    inverse = (x0 - x1*epsilon)/(x0^2)
    return inverse

def make_operations(a, b):
    # Build group law and scalar multiplication for an elliptic curve
    # y^2 = x^3 + a*x + b

    def plus(P1, P2):
        # Group law from Bosma & Lenstra.
        # Assumes P1 - P2 has odd order,
        # which is always true if the elliptic curve has odd order.
        X1, Y1, Z1 = P1
        X2, Y2, Z2 = P2
        X3 = (X1*Y2 + X2*Y1)*(Y1*Y2 - a*(X1*Z2 + X2*Z1) - 3*b*Z1*Z2) \
             - (Y1*Z2 + Y2*Z1)*(a*X1*X2 + 3*b*(X1*Z2 + X2*Z1) - a^2*Z1*Z2)
        Y3 = (Y1*Y2 + a*(X1*Z2 + X2*Z1) + 3*b*Z1*Z2)*(Y1*Y2 - a*(X1*Z2 + X2*Z1) - 3*b*Z1*Z2) \
             + (3*X1*X2 + a*Z1*Z2)*(a*X1*X2 + 3*b*(X1*Z2 + X2*Z1) - a^2*Z1*Z2)
        Z3 = (Y1*Z2 + Y2*Z1)*(Y1*Y2 + a*(X1*Z2 + X2*Z1) + 3*b*Z1*Z2) \
             + (X1*Y2 + X2*Y1)*(3*X1*X2 + a*Z1*Z2)
        return [X3, Y3, Z3]

    def mult(m, P):
        res = [0, 1, 0]  # Point at infinity
        # Basic double-and-add loop
        for bit in reversed(Integer(m).digits(2)):
            res = plus(res, res)
            if bit == 1:
                res = plus(res, P)
        return res

    def equals(P1, P2):
        # Projective equality:
        X1, Y1, Z1 = P1
        X2, Y2, Z2 = P2

        # These will all be zero iff X2 = c*X1, Y2 = c*X2, Z1 = c*Z2 for some nonzero c
        minors = [X1*Y2 - X2*Y1, X1*Z2 - X2*Z1, Y1*Z2 - Y2*Z1]

        return (minors[0] == 0) and (minors[1] == 0) and (minors[2] == 0)

    return plus, mult, equals

# Virat

Parameters

In [16]:
#Parameters

#prime p
p = 103
Fp = GF(p)

#curve parameters
a = 24
b = 87

#elliptic curve
E_p = EllipticCurve(Fp, [a, b])

#generator point P = [p] * (P hat)
P = E_p(1, 3)
print(P[2])


#Check if the generator point is on the curve
if P in E_p:
    print("Generator point P is on the curve.")

#order q of the generator, q prime
q = P.order()
print("Order of the generator:", q)

R.<t> = PolynomialRing(Fp)
S = R.quotient(t^2, 'epsilon')
epsilon = S.gen()

E_eps = EllipticCurve(S, [24,87])

P2 = E_eps(1, 3)
print(P2)


plus, mult, equals = make_operations(a, b)

1
Generator point P is on the curve.
Order of the generator: 107
(1 : 3 : 1)


Cryptosystem implementation

In [17]:
def key_generation():
    GF_q = FiniteField(q)
    x = GF_q.random_element()
    Y = mult(x, P)
    return (Y, x)

Y, x = key_generation()

In [18]:
def encrypt(Y, m):
    GF_q = FiniteField(q)
    r = GF_q.random_element()
    x_0, y_0, _ = E_p.random_point()
    tmp = 2 * y_0
    inv_y0 = inverse_mod(int(tmp), p)
    y_1 = (((3 * x_0 * x_0 * m * inv_y0) % p) + ((a * m * inv_y0) % p)) % p 
    M_hat = E_eps(x_0 + epsilon * m, y_0 + epsilon * y_1)
    C_1 = mult(r, P)
    C_2 = plus(M_hat, mult(r, Y))
    return (C_1, C_2)

C = encrypt(Y, 3)

In [19]:
def decrypt(x, C):
    C_1, C_2 = C
    M_hat = plus(C_2, mult((-1) * x, C_1))
    X, Y, Z = M_hat
    inv_Z = invert(Z)
    X = X * inv_Z
    Y = Y * inv_Z
    _ , coeff_eps = X.list()
    return coeff_eps
    
decrypt(x, C)

3

# Joye-Libert

Parameters

In [23]:
#Parameters

#prime p
p = 103
Fp = GF(p)

#curve parameters
a = 24
b = 87

#elliptic curve
E_p = EllipticCurve(Fp, [a, b])

#generator point P = [p] * (P hat)
P = E_p(1, 3)
print(P[2])


#Check if the generator point is on the curve
if P in E_p:
    print("Generator point P is on the curve.")

#order q of the generator, q prime
q = P.order()
print("Order of the generator:", q)

1
Generator point P is on the curve.
Order of the generator: 107


In [2]:
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
print(p%4)
K = GF(p)
a = 0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
E = EllipticCurve(K, (a, b))
P = E(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
q = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551
E.set_order(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 * 0x1)

3


In [3]:
plus, mult, equals = make_operations(a, b)

Functions for the cryptosystem

In [43]:
def gamma(Q):
    #print(Q)
    x, y, z = Q
    x = int(x)
    y = int(y)
    z = int(z)
    assert(z % p == 0)
    inv_y = inverse_mod(y, p**2)
    x = (x * inv_y) % (p**2)
    y = (y * inv_y) % (p**2)
    assert(y == 1)
    assert(x % p == 0)
    return x // p

def compute_class(Q):
    x, y, z = Q
    x = int(x)
    y = int(y)
    z = int(z)
    inv_q = inverse_mod(q, p)
    a = mult(q, Q)
    return ((gamma(a) * inv_q) % p)

def delta(Q):
    x, y, _ = Q
    x = int(x)
    y = int(y)
    return ((x ** 3 + a * x + b - y ** 2) % (p ** 2)) // p   

def little_psi(Q):
    x, y, _ = Q
    x = int(x)
    y = int(y)
    delta_Q = delta(Q)
    inv_y = inverse_mod(2 * y, p)
    return (delta_Q * inv_y) % p

def capital_psi(Q):
    x, y, z = Q
    R = Integers(p*p)
    x = R(x)
    y = R(y)
    z = R(z)
    if z == 0:
        return Q
    inv_z = 1/z
    x = (x * inv_z) 
    y = (y * inv_z) 
    z = (z * inv_z)
    Q = x, y, z
    return [x, y + little_psi(Q) * p, 1] #z=1 here

Test cell

In [7]:
GF_p = FiniteField(p)
x = GF_p.random_element()
print(1/x)

83424859242440795648229829882356891420437859070267581371895681571200318530986


Cryptosystem implementation

In [44]:
def key_generation():
    GF_q = FiniteField(q)
    x = GF_q.random_element()
    Y = mult(x, P)
    return (Y, x)

Y, x = key_generation()
print(Y)

[75646141355366343745554371254268106501053173153496267871337478615039239872550, 8098162052085318002550947114596255420418289247432173302464416257368120714251, 101676398661387027137539277256032579614944329471726542900325345953925225202659]


Additive cryptosystem

In [18]:
def encrypt_additive(Y, m):
    GF_q = FiniteField(q)
    r = GF_q.random_element()
    C_1 = mult(r, P)
    C_2 = mult(r, Y)
    C_2_tilde = capital_psi(C_2)
    beta = compute_class(C_2_tilde)
    c_2 = (m + beta) % p
    return (C_1, c_2)

C = encrypt_additive(Y, 4)
print(C)


def add_constant_to_encryption(Y, m, K):
    C_1, c_2 = encrypt_additive(Y,m)
    c_2 = (c_2 + K) % p
    return (C_1, c_2)

([85744128889824871292179817482484383304824244218036678891867301056350096551951, 48031250024592932947226771944246270087775973130292993526977001949086834148123, 89331426318170280312379239845692319485247905745834263708111473054243824231154], 4)


In [19]:
def decrypt_additive(x, C):
    C_1, c_2 = C
    C_1_times_x = mult(x, C_1)
    t = capital_psi(C_1_times_x)
    v = compute_class(t)
    return (c_2 - v) % p

print(decrypt_additive(x, C))

4


Multiplicative cryptosystem

In [45]:
def encrypt_multiplicative(Y, m):
    GF_q = FiniteField(q)
    r = GF_q.random_element()
    C_1 = mult(r, P)
    C_2 = mult(r, Y)
    C_2_tilde = capital_psi(C_2) 
    beta = compute_class(C_2_tilde)
    c_2 = (m * beta) % p
    return (C_1, c_2)

C = encrypt_multiplicative(Y, 4)
print(C)


def multiply_constant_to_encryption(Y, m, K):
    C_1, c_2 = encrypt_multiplicative(Y,m)
    c_2 = (c_2 * K) % p
    return (C_1, c_2)

([113542551223302214802109765626557691283352831652057301962086130014434271126304, 83991012978463443074893682919307591681403800267416906571579121601264967444375, 86383317485041015098807706706535610994742780750898350496457588733755623218465], 100413571199829182654995394563593012091375767022896089211322763799245423346759)


In [46]:
def decrypt_multiplicative(x, C): # DOESN'T DECRYPT WELL!!!!
    C_1, c_2 = C
    C_1_times_x = mult(x, C_1)
    t = capital_psi(C_1_times_x)
    v = compute_class(t)
    return (c_2 / v) % p

print(decrypt_multiplicative(x, C))

12773725122042835794778916777761283441627415386920406590409437174704774100252
